In [1]:
# TODO: Fill this in based on where you saved the training and testing data
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from alexnet import AlexNet
import pickle
import numpy as np
import pandas as pd
from scipy.misc import imread




training_file = 'train.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)

X_train, y_train = train['features'], train['labels']

print(X_train.shape, y_train.shape)

(39209, 32, 32, 3) (39209,)


In [2]:
# TODO: Split data into training and validation sets.
X_train, X_validation, y_train, y_validation = train_test_split(
    X_train, y_train, test_size=0.20, random_state=42)

print('Training set split')
print(X_train.shape, y_train.shape)

Training set split
(31367, 32, 32, 3) (31367,)


In [3]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

In [4]:
# TODO: Define placeholders and resize operation.
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
resized = tf.image.resize_images(x, (227, 227))

sign_names = pd.read_csv('signnames.csv')
nb_classes = 43

one_hot_y = tf.one_hot(y, 43)
rate = 0.0015

EPOCHS = 1
BATCH_SIZE = 128

In [5]:
fc7 = AlexNet(resized, feature_extract=True)
fc7 = tf.stop_gradient(fc7)

In [6]:
# TODO: Add the final layer for traffic sign classification.
shape = (fc7.get_shape().as_list()[-1], nb_classes)  # use this shape for the weight matrix
fc9_w = tf.Variable(tf.truncated_normal(shape=shape, stddev=1e-2))
fc9_b = tf.Variable(tf.zeros(nb_classes))
logits = tf.nn.xw_plus_b(fc7, fc9_w, fc9_b)


In [7]:
# TODO: Define loss, training, accuracy operations.
probs = tf.nn.softmax(logits)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
tf.summary.scalar('loss', loss_operation)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)


##evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy_operation)
summary_op = tf.summary.merge_all()


In [8]:
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    num_examples = 200
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [9]:
# TODO: Train and evaluate the feature extraction model.
with tf.Session() as sess:
    writer = tf.summary.FileWriter('data/tensorboard-logs/', sess.graph)
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)    
    num_examples = 200
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            summary, training = sess.run([summary_op,training_operation], feed_dict={x: batch_x, y: batch_y})
            writer.add_summary(summary)
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
    writer.flush()
    try:
        saver
    except NameError:
        saver = tf.train.Saver()
    saver.save(sess, 'lenet')
    print("Model saved")


Training...

EPOCH 1 ...
Validation Accuracy = 0.270

Model saved
